Jonathan Schultz and Trinh Pham

**IMBD Dataset**


There are a set of 50,000 polarized reviews, split into half (25,000) for testing. Reviews have been converted into integers.

In [ ]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
            num_words=10000)
# Importing libraries from TensorFlow and Keras

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
#list of reviews of word indices, 1s = postive 0s = negatives
train_data[0]
train_labels[0]

1

In [ ]:
#restricts to the top 10,000 frequent words, it should not exceed 10,000
max([max(sequence) for sequence in train_data])

9999

In [ ]:
word_index = imdb.get_word_index()   #dictionary mapping
reverse_word_index = dict(
 [(value, key) for (key, value) in word_index.items()]) #revereses and maps indices to words
decoded_review = " ".join(
 [reverse_word_index.get(i - 3, "?") for i in train_data[0]])  #decodes review. Offset by 3


1641221/1641221 [==============================] - 0s 0us/step


In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
 results = np.zeros((len(sequences), dimension))        #creates all-zero matrix
 for i, sequence in enumerate(sequences):
    for j in sequence:
      results[i, j] = 1.                            # Sets specific indices of result[i] to 1s
 return results
x_train = vectorize_sequences(train_data)       # Vectorized train data
x_test = vectorize_sequences(test_data)   #vetorized training data

In [ ]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [ ]:
# Vectorized labels, which then will be ready to be fed into a neural network

y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

In [ ]:
# model definition

from tensorflow import keras
from tensorflow.keras import layers


# Each dense layer is number of units layer. Each layer with 16 units and ReLU is our activation function.

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")        # This is the ouput layer with a single neuron
])                                               # sigmoid is the activation function to get a probability (between 0 and 1).

In [ ]:
#binary_crossentropy loss function
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Validation set to monitor accuracy of model

x_val = x_train[:10000]               #  Passes 10,000 samples from x_train to x_val
partial_x_train = x_train[10000:]     # Passes 10,0000 samples to partial_x_trail
y_val = y_train[:10000]               # Asssign first 10,000 samples from y_train to y_val
partial_y_train = y_train[10000:]

In [ ]:
#Model training
history = model.fit(partial_x_train,  #returns historic object
                    partial_y_train,
                    epochs=20,            #20 iterations
                    batch_size=512,       #mini-batches of 512 samples
                    validation_data=(x_val, y_val))

In [ ]:
#model.fit() returns history object
#history object is a dictionary containing data
history_dict = history.history
history_dict.keys()

In [ ]:
#Plotting the training and validation loss
import matplotlib.pyplot as plt #Import pyplot module to visualize data
history_dict = history.history #Extract the history
loss_values = history_dict ["loss"] #Get the loss values
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values)+ 1)
plt.plot (epochs, loss_values, "bo", label= "Training loss")
plt.plot (epochs, val_loss_values, "b", label= "Validation loss")
plt.title("Training and validation loss") #Set the title for the plot
plt.xlabel("Epochs") #x-axis as "Epochs"
plt.ylabel("Loss") #y-axis as "Loss"
plt.legend()
plt.show()

In [ ]:
# Plotting the training and validation accuracy
plt.clf()
acc = history_dict["accuracy"] #Get the accuracy values from the training
val_acc = history_dict["val_accuracy"] #Get the  validation accuracy values
plt.plot(epochs, acc, "bo", label="Training acc") #Plot the training accuracy values
plt.plot(epochs, val_acc, "b", label="Validation acc") # Plot the validation accuracy values
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
model = keras.Sequential([ #Define the sequential model
    layers.Dense(16, activation="relu"), # Add the first dense layer with 16 neurons and 'relu' activation
    layers.Dense(16, activation="relu"), # Adding second dense layer with 16 neurons
  layers.Dense(1, activation="sigmoid")  #Add the output layer with 1 neuron and 'sigmoid' activation
])
model.compile(optimizer="rmsprop", # Compile the model
              loss="binary_crossentropy", # binary crossentropy is a common loss function
              metrics=["accuracy"])
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)


In [ ]:
model.predict(x_test)

# Most Streamed Spotify Songs DL model

The dataset has a comprehensive list of famous songs of 2023

The dataset includes information such as track name, artist's name, release date, Spotify playlists and charts, streaming stats, and music streaming presence

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset encode it
data = pd.read_csv('/content/spotify-2023.csv', encoding='ISO-8859-1')

# Convert the 'streams' column to integers
data = data[data['streams'].str.isnumeric()]
data['streams'] = data['streams'].astype(int)


# Drop the 'key' and 'mode' columns since these are colummns that contain strings
data = data.drop(['key', 'mode'], axis=1)

# Define features and target variable
X = data[['bpm', 'energy_%', 'danceability_%', 'valence_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']]
y = (data['streams'] > 2800000000).astype(int)  # 1 if streams > 2.8 trillion, 0 otherwise

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='relu')) # Adding a dense layer with 12 neurons
model.add(Dense(8, activation='relu')) # Adding another dense layer with 8 neurons with the activation function as Relu
model.add(Dense(1, activation='sigmoid')) # Output layer with Sigmoid function

# Compile the model, loss = binary_crossentropy, optimizer = adam
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=10)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy*100:.2f}%")
# This model had an accuracy of 98.95% in predicting whether a song would be able to reach the top charts. I think this model is semi accurate
# because it is guranteed a song would be viral if it accures over 2.8 trillion streams. This was the amount that one of the highest streamed songs accured this year.
# In general i do not think this model is reliable because their is no clear target amount of streams for what it takes to reach the charts.
# The data is also based on data from songs that are already viral

Epoch 1/50
77/77 [==============================] - 4s 3ms/step - loss: 0.6309 - accuracy: 0.6597
Epoch 2/50
77/77 [==============================] - 0s 3ms/step - loss: 0.3520 - accuracy: 0.9777
Epoch 3/50
77/77 [==============================] - 0s 3ms/step - loss: 0.1737 - accuracy: 0.9921
Epoch 4/50
77/77 [==============================] - 0s 3ms/step - loss: 0.0860 - accuracy: 0.9947
Epoch 5/50
77/77 [==============================] - 0s 3ms/step - loss: 0.0531 - accuracy: 0.9947
Epoch 6/50
77/77 [==============================] - 0s 3ms/step - loss: 0.0405 - accuracy: 0.9947
Epoch 7/50
77/77 [==============================] - 0s 3ms/step - loss: 0.0347 - accuracy: 0.9947
Epoch 8/50
77/77 [==============================] - 0s 3ms/step - loss: 0.0317 - accuracy: 0.9947
Epoch 9/50
77/77 [==============================] - 0s 3ms/step - loss: 0.0297 - accuracy: 0.9947
Epoch 10/50
77/77 [==============================] - 0s 3ms/step - loss: 0.0285 - accuracy: 0.9947
Epoch 11/50
77/77 [

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data = pd.read_csv('/content/spotify-2023.csv', encoding='ISO-8859-1')

# Convert the 'streams' column to integers
data = data[data['streams'].str.isnumeric()]
data['streams'] = data['streams'].astype(int)


# Drop the 'key' and 'mode' columns
data = data.drop(['key', 'mode'], axis=1)

# Define features and target variable
X = data[['bpm', 'energy_%', 'danceability_%', 'valence_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']]
y = (data['streams'] > 2800000000).astype(int)  # 1 if streams > 2.8 trillion, 0 otherwise

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='hinge', optimizer='sgd', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=10)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy*100:.2f}%")
# This is the same code from above but we used a differnt loss, optimizer, and activiation function
# It improved the accuracy by .53% which is quiet significant since it was already close to 100%
# These functions must have worked better with out data and features selected

Epoch 1/50
77/77 [==============================] - 3s 8ms/step - loss: 1.3340 - accuracy: 0.9947
Epoch 2/50
77/77 [==============================] - 1s 7ms/step - loss: 1.2279 - accuracy: 0.9947
Epoch 3/50
77/77 [==============================] - 0s 6ms/step - loss: 1.1596 - accuracy: 0.9947
Epoch 4/50
77/77 [==============================] - 1s 8ms/step - loss: 1.1170 - accuracy: 0.9947
Epoch 5/50
77/77 [==============================] - 1s 7ms/step - loss: 1.0894 - accuracy: 0.9947
Epoch 6/50
77/77 [==============================] - 0s 6ms/step - loss: 1.0711 - accuracy: 0.9947
Epoch 7/50
77/77 [==============================] - 0s 4ms/step - loss: 1.0581 - accuracy: 0.9947
Epoch 8/50
77/77 [==============================] - 0s 3ms/step - loss: 1.0485 - accuracy: 0.9947
Epoch 9/50
77/77 [==============================] - 0s 3ms/step - loss: 1.0414 - accuracy: 0.9947
Epoch 10/50
77/77 [==============================] - 0s 3ms/step - loss: 1.0359 - accuracy: 0.9947
Epoch 11/50
77/77 [

# Airline Passenger Satsifaction

This dataset takes into consideration all of factors that correlate to passenger satsifaction by flight distance, wifi service, ease of online booking, foods, online boarding, seat comfort, inflight entertainment, baggage handling, cleanliness, etc.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#uploading files
df = pd.read_csv('test.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976 entries, 0 to 25975
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         25976 non-null  int64  
 1   id                                 25976 non-null  int64  
 2   Gender                             25976 non-null  object 
 3   Customer Type                      25976 non-null  object 
 4   Age                                25976 non-null  int64  
 5   Type of Travel                     25976 non-null  object 
 6   Class                              25976 non-null  object 
 7   Flight Distance                    25976 non-null  int64  
 8   Inflight wifi service              25976 non-null  int64  
 9   Departure/Arrival time convenient  25976 non-null  int64  
 10  Ease of Online booking             25976 non-null  int64  
 11  Gate location                      25976 non-null  int

In [ ]:
#Dropping the unnecessary column
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('id', axis=1)

df.dropna()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Female,Loyal Customer,52,Business travel,Eco,160,5,4,3,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,Female,Loyal Customer,36,Business travel,Business,2863,1,1,3,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,Male,disloyal Customer,20,Business travel,Eco,192,2,0,2,4,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,Male,Loyal Customer,44,Business travel,Business,3377,0,0,0,2,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,4,3,...,2,2,2,2,4,2,4,0,20.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,Male,disloyal Customer,34,Business travel,Business,526,3,3,3,1,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
25972,Male,Loyal Customer,23,Business travel,Business,646,4,4,4,4,...,4,4,5,5,5,5,4,0,0.0,satisfied
25973,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,1,5,...,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
25974,Male,Loyal Customer,14,Business travel,Business,1127,3,3,3,3,...,4,3,2,5,4,5,4,0,0.0,satisfied


#Data Preprocessing

**Label Encoder**

will be using Label Encoder to change any categorical column into 0s and 1s for machine readability

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

columns = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']
for col in columns:
  df[col] = le.fit_transform(df[col])

df

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,0,52,0,1,160,5,4,3,4,...,5,5,5,5,2,5,5,50,44.0,1
1,0,0,36,0,0,2863,1,1,3,1,...,4,4,4,4,3,4,5,0,0.0,1
2,1,1,20,0,1,192,2,0,2,4,...,2,4,1,3,2,2,2,0,0.0,0
3,1,0,44,0,0,3377,0,0,0,2,...,1,1,1,1,3,1,4,0,6.0,1
4,0,0,49,0,1,1182,2,3,4,3,...,2,2,2,2,4,2,4,0,20.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,1,1,34,0,0,526,3,3,3,1,...,4,3,2,4,4,5,4,0,0.0,0
25972,1,0,23,0,0,646,4,4,4,4,...,4,4,5,5,5,5,4,0,0.0,1
25973,0,0,17,1,1,828,2,5,1,5,...,2,4,3,4,5,4,2,0,0.0,0
25974,1,0,14,0,0,1127,3,3,3,3,...,4,3,2,5,4,5,4,0,0.0,1


**Normal Stadardization**

for scaling and transforming features in our dataset. Transforms data in a way to make a mean of 0 and a standard deviation of 1. Helps with the data training process.
Here I am scaling columns 7 - end because those are the columms that needs to be interpreted in ways that machine can understand.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select columns to be scaled (from the 7th column to the second-to-last column)
columns_to_scale =df.columns[2:3].tolist() + df.columns[5:-1].tolist()

# Create a DataFrame with only selected columns
X_selected = df[columns_to_scale]

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform scaler on selected columns
X_scaled = scaler.fit_transform(X_selected)

# Replace scaled values back into the original DataFrame
df[columns_to_scale] = X_scaled
df

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,0,0.817887,0,1,-1.035171,1.703853,0.621641,0.172143,0.797831,...,1.227138,1.259170,1.250974,1.161716,-1.035348,1.144063,1.298998,0.953810,0.779894,1
1,0,0,-0.239238,0,0,1.671443,-1.291598,-1.334871,0.172143,-1.542065,...,0.479907,0.479178,0.492730,0.311739,-0.247517,0.297077,1.298998,-0.382286,-0.392913,1
2,1,1,-1.296363,0,1,-1.003128,-0.542735,-1.987042,-0.535609,0.797831,...,-1.014556,0.479178,-1.782001,-0.538238,-1.035348,-1.396893,-0.974927,-0.382286,-0.392913,0
3,1,0,0.289325,0,0,2.186131,-2.040460,-1.987042,-1.951114,-0.762100,...,-1.761787,-1.860800,-1.782001,-2.238193,-0.247517,-2.243878,0.541023,-0.382286,-0.232985,1
4,0,0,0.619676,0,1,-0.011804,-0.542735,-0.030530,0.879895,0.017866,...,-1.014556,-1.080807,-1.023758,-1.388216,0.540315,-1.396893,0.541023,-0.382286,0.140181,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,1,1,-0.371378,0,0,-0.668681,0.206128,-0.030530,0.172143,-1.542065,...,0.479907,-0.300815,-1.023758,0.311739,0.540315,1.144063,0.541023,-0.382286,-0.392913,0
25972,1,0,-1.098152,0,0,-0.548521,0.954990,0.621641,0.879895,0.797831,...,0.479907,0.479178,1.250974,1.161716,1.328146,1.144063,0.541023,-0.382286,-0.392913,1
25973,0,0,-1.494573,1,1,-0.366278,-0.542735,1.273812,-1.243362,1.577797,...,-1.014556,0.479178,-0.265514,0.311739,1.328146,0.297077,-0.974927,-0.382286,-0.392913,0
25974,1,0,-1.692784,0,0,-0.066878,0.206128,-0.030530,0.172143,0.017866,...,0.479907,-0.300815,-1.023758,1.161716,0.540315,1.144063,0.541023,-0.382286,-0.392913,1


#Dataset training and testing

Now we are training and testing the **datasets**

In [ ]:
from typing_extensions import dataclass_transform
from sklearn.model_selection import train_test_split

# Include all columns except the target variable
X = df.drop(columns=['satisfaction'])
y= df['satisfaction']

# Split data into training and testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.4, random_state = 0)          # We are taking 60% of the dataset to train, 20% for test and another 20% for validation.
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 0)

In [ ]:
# Define the model
from tensorflow import keras
from tensorflow.keras import layers

# We have three lines of 'layer.Dense...' code because we are adding three layers,
model = keras.Sequential([
    layers.Dense(6, activation="relu", input_shape=(22,)),  #Input shape of 23 because we are excluding the satsifaction column
    layers.Dense(6, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [ ]:
# Assign Loss function and Optimizer
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Define the model
history = model.fit(X_train,
                    y_train,
                    epochs=25,
                    batch_size=16,
                    validation_split=0.2,
                    verbose=1
)

Epoch 1/25
780/780 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.5595 - val_loss: nan - val_accuracy: 0.5675
Epoch 2/25
780/780 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.5595 - val_loss: nan - val_accuracy: 0.5675
Epoch 3/25
780/780 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.5595 - val_loss: nan - val_accuracy: 0.5675
Epoch 4/25
780/780 [==============================] - 3s 4ms/step - loss: nan - accuracy: 0.5595 - val_loss: nan - val_accuracy: 0.5675
Epoch 5/25
780/780 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.5595 - val_loss: nan - val_accuracy: 0.5675
Epoch 6/25
780/780 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.5595 - val_loss: nan - val_accuracy: 0.5675
Epoch 7/25
780/780 [==============================] - 2s 3ms/step - loss: nan - accuracy: 0.5595 - val_loss: nan - val_accuracy: 0.5675
Epoch 8/25
780/780 [============================

#DL Model for diabetes prediction

This model uses a dataset that takes a number of patients. The data set takes in pregnancies, glucose level, blood pressure, insulin, BMI etc. into consideration when calculating for the likelihood of being diabetic

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset from CSV
data = pd.read_csv('/content/diabetes.csv')

# Split the data into features and target variable
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a Sequential model
model = Sequential()

# Add input layer with 12 neurons, input shape matching the number of features in X_train, and ReLU activation
model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))

# Add a hidden layer with 8 neurons and ReLU activation
model.add(Dense(8, activation='relu'))

# Add output layer with 1 neuron (binary outcome) and sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy loss for binary classification, Adam optimizer, and accuracy metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=10)

# Evaluate the model using test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy*100:.2f}%")

# This model has the ability to predict whether a patient based on several features, is a diabetic
# It has an accuracy of 76.62%

Epoch 1/10
62/62 [==============================] - 2s 6ms/step - loss: 0.7330 - accuracy: 0.4951
Epoch 2/10
62/62 [==============================] - 0s 4ms/step - loss: 0.6460 - accuracy: 0.6564
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 0.5912 - accuracy: 0.6938
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 0.5515 - accuracy: 0.7150
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 0.5227 - accuracy: 0.7313
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 0.5052 - accuracy: 0.7394
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 0.4916 - accuracy: 0.7557
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4816 - accuracy: 0.7557
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.7622
Epoch 10/10
5/5 [==============================] - 0s 4ms/step - loss: 0.5274 - accuracy: 0.7078
Model Accuracy: 70.78

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset from CSV
data = pd.read_csv('/content/diabetes.csv')

# Split the data into features and target variable
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a Sequential model
model = Sequential()

# Add input layer with 12 neurons, input shape matching the number of features in X_train, and tanh activation
model.add(Dense(12, input_dim=X_train.shape[1], activation='tanh'))

# Add a hidden layer with 8 neurons and tanh activation
model.add(Dense(8, activation='tanh'))

# Add output layer with 1 neuron (binary outcome) and sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with hinge for loss function, sgd optimizer, and accuracy metric
model.compile(loss='hinge', optimizer='sgd', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=10)

# Evaluate the model using test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy*100:.2f}%")

# This is the same model with different activation, loss, and optimizer functions.
# We now use a hinge loss functions and a stochastic gradient descent for the optimizer function. We also changed the activation function to tanh.
# These changes resulted in a 6% loss in accuracy.